In [ ]:
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from sklearn_pandas import DataFrameMapper
from sklearn import preprocessing
from sklearn.metrics import normalized_mutual_info_score
import itertools
import getpass
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.pipeline import Pipeline
from collections import namedtuple
from scipy import stats
# Display more rows and get rid of the margins
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Display mulitiple values from each cell
# https://stackoverflow.com/a/42476224
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from itertools import cycle

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

# Define paths and functions

In [ ]:
# Define the directories that we're going to read data from
data_dir = Path('data')
swarm_dir = Path('swarm_dir_bs2.0')

In [ ]:
def bal_samp(df, strata, balance, order, keys, n_splits=5, n_draws=100, drop_strata=False):
    """Balanced sampling across strata

    Parameters
    ----------
    df: pandas.DataFrame
        Dataframe from which you want to sample
    strata: str or list of str
        Name(s) of the column or columns that define the groups
        from which you want a balanced sample
    balance: str or list of str
        Name(s) of the columns or columns containing the factors
        you want to evenly sample across strata
    order: str or list of str
        Name(s) of the column whose distribution you want to preserve
    keys: list of str
        Name(s) of the column(s) that you will use to match
        the output back to your original column
    n_splits: int
        Number of cross validation folds you want to create per draw
    n_draws: int
        Number of balanced samples of your dataset you want to create

    Returns
    -------
    draws_df: pandas.DataFrame
        Dataframe with number of rows about equal to number of rows in df
        and number of columns equal to n_draws + len(keys) + len(strata) + len(balance) + len(order).
        Contains the crossfold labels for balanced sampling across the strata you've defined.
    """
    # create dict of minimum count in each strata of each combination of balance factor
    bal_dict = (df.groupby(strata + balance)[[keys[0]]]
                  .nunique()
                  .groupby(balance)
                  .min()
                  .to_dict('index'))
    bal_dict = {k: v[keys[0]] for k, v in bal_dict.items()}

    # Appologies for the disgusting nested loops
    # For each draw, at each strata level, for each unique combin
    draws_df = []
    # For each draw
    for nn in range(n_draws):
        strat_df = []
        # From each strata group
        for x, gbdf in df.groupby(strata):
            cvs_df = []
            # from each unique combination of balance values
            for bal_vals, num in bal_dict.items():
                # create an index selecting the rows at those balance values
                ind = np.ones((len(gbdf))).astype(bool)
                for bcol, bv in zip(balance, bal_vals):
                    ind = np.logical_and(ind, gbdf[bcol] == bv)
                if ind.sum() == 0:
                    if drop_strata:
                        print(f"There are no rows with {bal_vals} for {balance} in group {x}.")
                        continue
                    else:
                        raise ValueError(f"There are no rows with {bal_vals} for {balance} in group {x}.")
                    
                # draw a random sample of the group members
                # that meet the balance criteria
                # and sort them by the order values
                bal_df = gbdf[ind].sample(
                    n=num).sort_values(order).loc[:, keys]
                # create a list of the cross validation values long enough to match
                cv_inds = list(np.arange(n_splits))*((len(bal_df)//n_splits)+1)
                bal_df['draw_%d' % nn] = cv_inds[:len(bal_df)]
                # and append them to a list
                cvs_df.append(bal_df)
            # combine these lists to get all the rows for that strata
            # and append them to create a list of selected rows from all the strata
            strat_df.append(pd.concat(cvs_df).loc[:, ['draw_%d' % nn]])
        # pull these all together to create the draws dataframe
        draws_df.append(pd.concat(strat_df))
    draws_df = pd.concat(draws_df, axis=1)
    # Merge back in the indicator variables
    draws_df = (df.loc[:,
                       keys
                       + strata
                       + balance
                       + order]
                .merge(draws_df,
                       right_index=True,
                       left_index=True,
                       how='left')
                )
    # make sure the shape is still ok
    assert draws_df.shape[0] == df.shape[0]
    assert draws_df.shape[1] == (n_draws
                                 + len(keys)
                                 + len(strata)
                                 + len(balance)
                                 + len(order))
    return draws_df


def gen_binned_perms(df, bin_levels, n_perms=1000, boot=False):
    df = df.copy(deep=True)
    permed_inds = []
    permed_inds.append(df.index.values)
    if not 'ind' in df.columns:
        df['ind'] = df.index.values
    else:
        raise Exception
    for pn in range(n_perms):
        if not boot:
            permed_inds.append(df.groupby(bin_levels).ind.transform(
                np.random.permutation).values)
        else:
            permed_inds.append(df.sample(frac=1, replace=True).ind.values)
    return permed_inds


def get_cols(raw_df):
    base_meta_cols = ['contrast', 'fmri_beta_gparc_numtrs', 'fmri_beta_gparc_tr', 'lmt_run',
                      'mid_beta_seg_dof', 'task', 'collection_id', 'dataset_id', 'subjectkey',
                      'src_subject_id', 'interview_date', 'interview_age',
                      'gender', 'event_name', 'visit', 'rsfm_tr', 'eventname',
                      'rsfm_nreps', 'rsfm_numtrs', 'pipeline_version',  'scanner_manufacturer_pd',
                      'scanner_type_pd', 'mri_info_deviceserialnumber', 'magnetic_field_strength',
                      'procdate', 'collection_title', 'promoted_subjectkey', 'study_cohort_name',
                      'ehi_ss_score', '_merge', 'qc_ok', 'age_3mos', 'abcd_betnet02_id', 'fsqc_qc',
                      'rsfmri_cor_network.gordon_visitid',
                      'mrirscor02_id',  'site_id_l', 'mri_info_manufacturer',
                      'mri_info_manufacturersmn', 'mri_info_deviceserialnumber',
                      'mri_info_magneticfieldstrength', 'mri_info_softwareversion',
                      'unique_scanner', 'tbl_id', 'tbl_visitid', 
                     'modality', 'metric', 'source_file', 'tbl_id_y', 'source_file_y', 
                     'run', 'mri_info_visitid', 'dmri_dti_postqc_qc',
       'iqc_t2_ok_ser', 'iqc_mid_ok_ser', 'iqc_sst_ok_ser',
       'iqc_nback_ok_ser', 'tfmri_mid_beh_perform.flag',
       'tfmri_nback_beh_perform.flag', 'tfmri_sst_beh_perform.flag',
       'tfmri_mid_all_beta_dof', 'tfmri_mid_all_sem_dof',
       'tfmri_sst_all_beta_dof', 'tfmri_sst_all_sem_dof',
       'tfmri_nback_all_beta_dof', 'tfmri_nback_all_sem_dof',
       'mrif_score', 'mrif_hydrocephalus', 'mrif_herniation',
       'mr_findings_ok', 'tbl_numtrs', 'tbl_dof', 'tbl_nvols', 'tbl_tr', 'tbl_subthresh.nvols',
                     'rsfmri_cor_network.gordon_tr', 'rsfmri_cor_network.gordon_numtrs',
       'rsfmri_cor_network.gordon_nvols',
       'rsfmri_cor_network.gordon_subthresh.nvols',
       'rsfmri_cor_network.gordon_subthresh.contig.nvols',
       'rsfmri_cor_network.gordon_ntpoints', 'dataset_id_y', 'tbl_mean.motion', 'tbl_mean.trans', 'tbl_mean.rot',
       'tbl_max.motion', 'tbl_max.trans', 'tbl_max.rot']
    meta_cols = raw_df.columns[raw_df.columns.isin(base_meta_cols)].values
    metric_cols = raw_df.columns[~raw_df.columns.isin(base_meta_cols)].values
    return metric_cols, meta_cols

In [ ]:
def big_sites(size_limit, df, metric_cols):
    notnull_mask = np.logical_and(~(df.loc[:, metric_cols].isnull().sum(1).astype(bool)),
                                  np.isfinite(df.loc[:, metric_cols]).sum(1).astype(bool))
    notnull_mask = np.logical_and(
        notnull_mask, df.unique_scanner.notnull())
    notnull_mask = np.logical_and(notnull_mask, df.qc_ok == 1)
    # for right now, just focus on sites with more than size_limit scans
    scans_per_sn = df.loc[notnull_mask, :].groupby(['unique_scanner'])[
        ['collection_id']].count().sort_values('collection_id', ascending=False)
    big_sns = df.loc[df.unique_scanner.isin(scans_per_sn.query('collection_id > @size_limit').index)
                     & notnull_mask, :]
    scans_per_bigsn = big_sns.groupby('unique_scanner')[
        ['collection_id']].count().sort_values('collection_id', ascending=False)

    print(f"Number of sites with more than {size_limit} scans:", len(
        big_sns.unique_scanner.unique()))
    print(f"Number of scans collected at sites with more than {size_limit} scans:", len(
        big_sns))
    print(f"Number of subjects collected at sites with more than {size_limit} scans:",
        big_sns.subjectkey.nunique())
    scans_per_bigsn.query('collection_id > @size_limit')
    return big_sns


def drop_bad(df, metric_cols, val_range=None, std_range=None, pct_range=None, verbose=False):
    if verbose:
        df = df.copy()
        df['cr'] = df.contrast + df.run
        full_data_subs = len(pd.unique((df.groupby('subjectkey')[['cr']].nunique() == df.groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))

    notnull_mask = np.logical_and(~(df.loc[:, metric_cols].isnull().sum(1).astype(bool)),
                                  np.isfinite(df.loc[:, metric_cols]).sum(1).astype(bool))
    dropped_sum = (~notnull_mask).sum()
    if verbose:        
        dropped_subs = full_data_subs - len(pd.unique((df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique() == df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))
        print(f"{dropped_sum} contrasts dropped for null data in one of the metrics")
        print(f"{dropped_subs} subjects dropped for null data in one metric on one contrast")
        full_data_subs = len(pd.unique((df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique() == df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))
    assert notnull_mask.sum() > 0
        
    notnull_mask = np.logical_and(
        notnull_mask, df.unique_scanner.notnull())
    if verbose:
        dropped_subs = full_data_subs - len(pd.unique((df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique() == df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))
        print(f"{(~notnull_mask).sum() - dropped_sum} contrasts dropped for not having a scanner id")
        print(f"{dropped_subs} subjects dropped for not having a scanner id")
        full_data_subs = len(pd.unique((df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique() == df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))
    dropped_sum = (~notnull_mask).sum()
    assert notnull_mask.sum() > 0
        
    notnull_mask = np.logical_and(notnull_mask, df.qc_ok == 1)
    if verbose:
        dropped_subs = full_data_subs - len(pd.unique((df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique() == df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))
        print(f"{(~notnull_mask).sum() - dropped_sum} contrasts dropped for failing QC")
        print(f"{dropped_subs} subjects dropped because one run failed QC")
        full_data_subs = len(pd.unique((df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique() == df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))
    dropped_sum = (~notnull_mask).sum()
    assert notnull_mask.sum() > 0

    if val_range is not None:
        notnull_mask = np.logical_and(notnull_mask, (np.abs(
            df.loc[:, metric_cols]) < val_range).product(1).astype(bool))
        if verbose:
            dropped_subs = full_data_subs - len(pd.unique((df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique() == df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))
            print(f"{(~notnull_mask).sum() - dropped_sum} contrasts dropped for exceeding value range")
            print(f"{dropped_subs} subjects dropped because one contrast had a value exceeding value range")
            full_data_subs = len(pd.unique((df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique() == df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))
        dropped_sum = (~notnull_mask).sum()
        assert notnull_mask.sum() > 0

    if std_range is not None:
        notnull_mask = np.logical_and(notnull_mask, (np.abs(
            (df.loc[:, metric_cols] - df.loc[:, metric_cols].mean())/ df.loc[:, metric_cols].std()) < std_range).product(1).astype(bool))
        if verbose:
            dropped_subs = full_data_subs - len(pd.unique((df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique() == df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))
            print(f"{(~notnull_mask).sum() - dropped_sum} contrasts dropped for exceeding variance range")
            print(f"{dropped_subs} subjects dropped because one contrast had a value exceeding variance range")
            full_data_subs = len(pd.unique((df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique() == df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))
        dropped_sum = (~notnull_mask).sum()
        assert notnull_mask.sum() > 0

    if pct_range is not None:
        top_pct = (100-pct_range)/100
        bottom_pct = pct_range/100
        notnull_mask = np.logical_and(notnull_mask, 
            (df.loc[:, metric_cols] <= df.loc[:, metric_cols].quantile([top_pct]).iloc[0, :])
                .product(1)
                .astype(bool))
        notnull_mask = np.logical_and(notnull_mask, 
            (df.loc[:, metric_cols] >= df.loc[:, metric_cols].quantile([bottom_pct]).iloc[0, :])
                .product(1)
                .astype(bool))
        if verbose:
            dropped_subs = full_data_subs - len(pd.unique((df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique() == df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))
            print(f"{(~notnull_mask).sum() - dropped_sum} contrasts dropped for exceeding percentile range")
            print(f"{dropped_subs} subjects dropped because one contrast had a value exceeding percentile range")
            full_data_subs = len(pd.unique((df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique() == df.loc[notnull_mask, :].groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values))
        dropped_sum = (~notnull_mask).sum()
        assert notnull_mask.sum() > 0
    return df.loc[notnull_mask, :].copy(deep=True)



In [ ]:
def get_full_subjects(longtbl, modality, metric_range=None, **kwargs):
    
    for ii, mm in enumerate(longtbl.query('modality == @modality').metric.unique()):
        metric_df = longtbl.loc[((longtbl.modality == modality) & (longtbl.metric == mm))].copy()
        unused_cols = metric_df.columns[(metric_df.notnull().sum() == 0)].values
        metric_df.drop(unused_cols, axis=1, inplace=True)

        metric_cols, meta_cols = get_cols(metric_df)
        
        if metric_range is not None:
            try:
                metric_nb = drop_bad(metric_df, metric_cols, val_range = metric_range[mm], **kwargs)
            except KeyError:
                metric_nb = drop_bad(metric_df, metric_cols, **kwargs)
        else:
            metric_nb = drop_bad(metric_df, metric_cols, **kwargs)
        if ii == 0:
            full_subj = set(metric_nb.subjectkey.unique())
        else:
            full_subj = full_subj.intersection(metric_nb.subjectkey.unique())
    
    return full_subj

In [ ]:
def make_cmd_list(user, prange, modality, metrics, bundle=5, n_draws=25,
                  run=None, task=None, contrast=None,
                  out_base='/data/nielsond/abcd/nielson_abcd_2018/',
                  out_subdir='release11',
                  swarm_dir='/data/nielsond/abcd/nielson_abcd_2018/swarm_dir',
                  image='/data/MLDSST/singularity_images/abcd_tmp-2018-04-26-a7f5b7d6a3b4.img',
                  bootstrap=False,
                  ymap_file = 'yfit.pkz',
                  skip_completed=False):
    if (modality =='tfmri') & ((task is None) | (contrast is None)):
        raise ValueError('If modality is tfmri, then run, task and contrast mus be provided')
    swarm_dir = Path(swarm_dir)
    cmds = []
    job_n = 0
    rsync_in = ''#'rsync -avch /data/MLDSST/abcd/abcd_swarm_dir /lscratch/$SLURM_JOBID/; '
    cmd = rsync_in
    for pn in range(*prange):
        for metric in metrics:
            if bootstrap:
                bs_or_pn = 'bs'
            else:
                bs_or_pn = 'pn'
            if modality != 'tfmri':
                out_path = f'{out_base}/{out_subdir}/{bs_or_pn}-{pn:04d}_{modality}_{metric}.pkz'
            else:
                out_path = f'{out_base}/{out_subdir}/{bs_or_pn}-{pn:04d}_crt_{task}_{contrast}.pkz'
            if skip_completed and Path(out_path).exists():
                continue
            if not Path(out_path).exists():
                if bootstrap:
                    perm_file = swarm_dir/f'boots_{modality}.pkz'
                else:
                    perm_file = swarm_dir/f'perms_{modality}.pkz'
                ymap_file = swarm_dir/ymap_file
                if modality != 'tfmri':
                    input_file = swarm_dir/f'{modality}_{metric}.pkz'
                else:
                    input_file = swarm_dir/f'crt_{task}_{contrast}_beta.pkz '


                cmd_base = ('export SINGULARITY_BINDPATH="/gs3,/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data";'
                            + f' singularity exec -H ~/temp_for_singularity {image}'
                            + f" python {swarm_dir}/run_abcd_perm_new_draws.py {pn} {perm_file}"
                            + f" {ymap_file} {input_file}"
                            + f" {out_path} 30 --n_draws={n_draws}")
                if bootstrap:
                    cmd_base += ' --bootstrap ;'
                else:
                    cmd_base += ' ;'
                #rsync_out = f'rsync -ach {out_path} {local_path}; chown $USER:MLDSST {local_path}; '
                cmd +=  cmd_base #+ rsync_out
                job_n +=1
                if job_n == bundle:
                    cmds.append(cmd)
                    cmd = rsync_in
                    job_n = 0
    return cmds

#cmds = '\n'.join(cmds)
#swarm_file.write_text(cmds)

# Prep Dataframes for swarm jobs


## rs-fMRI data

In [ ]:
con_old = pd.read_pickle(data_dir/'con.pkz')

con = pd.read_pickle(data_dir/'con_2.0.pkz')
print(len(con))
# drop the subjects that are in the 1.1 release
# as well as those that don't pass qc, those without a entry for gender, and ambidexterous individiuals
con = con.loc[(~con.subjectkey.isin(con_old.subjectkey.unique())), :]
print(con.subjectkey.nunique())
con = con.loc[(con.qc_ok == True ), :]
print(con.subjectkey.nunique())
con = con.loc[(con.gender.notnull()), :]
print(con.subjectkey.nunique())
con = con.loc[(con.ehi_ss_score ==1), :]
print(con.subjectkey.nunique())
con['age_6mos'] = (con['interview_age'] // 6) * 6

# Drop lefties


# Select the biggest scanner from each site
biggest_scanners = (con
                    .groupby(['site_id_l', 'unique_scanner'])[['subjectkey']]
                    .nunique()
                    .reset_index()
                    .sort_values(['site_id_l', 'subjectkey'], ascending=[True, False])
                    .groupby(['site_id_l'])
                    .first().reset_index())
con = con.loc[con.unique_scanner.isin(biggest_scanners.unique_scanner), :]
print(len(con))

In [ ]:
# Process connectivity data
# If processed connectivity data doesn't exist, create it
big_con_file = swarm_dir/'big_con.pkz'
if not big_con_file.exists():
    print("No processed connectivity data file found. Generating a new one.")
    #con = pd.read_pickle(data_dir/'con.pkz')
    #con.loc[con.mri_info_manufacturersmn == 'DISCOVERY MR750', 'mri_info_manufacturersmn'] = 'AAAA DISCOVERY MR750'
    con_metric_cols, con_meta_cols = get_cols(con)
    big_con = big_sites(100, con, con_metric_cols)

    big_con.to_pickle(big_con_file)
else:
    big_con = pd.read_pickle(big_con_file)

# Look at counts in connectivity data
big_con.groupby(['site_id_l',
                 'mri_info_manufacturer',
                 'mri_info_manufacturersmn',
                 'mri_info_deviceserialnumber',
                 'mri_info_softwareversion',
                 'unique_scanner']).count()

In [ ]:
# Load permutations or generate them
perm_file = swarm_dir/'perms_con.pkz'
if not perm_file.exists():
    print("No connectivity permutation file found. Generating a new one.")
    # within bucket shuffle
    big_con['age_3mos'] = (big_con['interview_age'] // 3) * 3
    bin_levels = ['gender', 'ehi_ss_score', 'age_3mos']
    con_perms = gen_binned_perms(big_con, bin_levels, n_perms=2000)
    with open(perm_file.as_posix(), 'wb') as h:
        pickle.dump(con_perms, h)
else:
    with open(perm_file.as_posix(), 'rb') as h:
        con_perms = pickle.load(h)

# If ymapper doesn't exist, create it
# We're doing this outside the swarm to ensure that every worker has the same
# encoding of the categories
ymap_file = swarm_dir/'yfit_con.pkz'
if not ymap_file.exists():
    print("No connectivity ymapper file found. Generating a new one.")

    ymapper = DataFrameMapper([("unique_scanner",preprocessing.LabelEncoder())])
    yfit = ymapper.fit(big_con)
    with open(swarm_dir/'yfit_con.pkz', 'wb') as h:
        pickle.dump(yfit, h)
        
# Generate bootstraps
boot_file = swarm_dir/'boots_con.pkz'

if not boot_file.exists():
    print("No connectivity bootstrap file found. Generating a new one.")
    # within bucket shuffle
    bin_levels = []
    con_boots = gen_binned_perms(big_con, bin_levels, n_perms=2000, boot=True)[1:]
    with open(boot_file.as_posix(), 'wb') as h:
        pickle.dump(con_boots, h)
else:
    with open(boot_file.as_posix(), 'rb') as h:
        con_boots = pickle.load(h)

## task-based data

In [ ]:
longtbl_old = pd.read_pickle(data_dir/'long_table.pkz')
longtbl = pd.read_pickle(data_dir/'long_table_2.0.pkz')

longtbl['unique_scanner'] = longtbl.site_id_l + '__' + longtbl.mri_info_deviceserialnumber + '__' + longtbl.mri_info_softwareversion

print(longtbl.subjectkey.nunique())
longtbl = longtbl.loc[(~longtbl.subjectkey.isin(longtbl_old.subjectkey.unique())), :]
print(longtbl.subjectkey.nunique())
longtbl = longtbl.loc[(longtbl.ehi_ss_score == 1), :]
print(longtbl.subjectkey.nunique())
longtbl = longtbl.loc[longtbl.gender.notnull(), :]
print(longtbl.subjectkey.nunique())

tb_b = longtbl.loc[((longtbl.modality == 'tfmri') & (longtbl.metric == 'beta') & (longtbl.run != 'all'))].copy().reset_index(drop=True)

unused_cols = tb_b.groupby(['task','run','metric','contrast']).count().sum().index[tb_b.groupby(['task','run','metric', 'contrast']).count().sum() < 7000]

tb_b.drop(unused_cols, axis=1, inplace=True)

metric_cols, meta_cols = get_cols(tb_b)

In [ ]:
longtbl.groupby(['task','run','metric','contrast'])

### Ymapper

In [ ]:
# If ymapper doesn't exist, create it
# We're doing this outside the swarm to ensure that every worker has the same
# encoding of the categories
ymap_file = swarm_dir/f'yfit.pkz'
if not ymap_file.exists():
    ymapper = DataFrameMapper([("unique_scanner",preprocessing.LabelEncoder())])
    yfit = ymapper.fit(longtbl[longtbl.unique_scanner.notnull()])
    with open(ymap_file, 'wb') as h:
        pickle.dump(yfit, h)
else:
    with open(ymap_file, 'rb') as h:
        yfit = pickle.load(h)

In [ ]:
# with single run per subject
# For task based analysis limit to only subjects that have data across all tasks and runs
tb_b_nb = drop_bad(tb_b, metric_cols, verbose=True, pct_range=0.25)
tb_b_nb['cr'] = tb_b_nb.contrast + tb_b_nb.run
# Get the first complete run for each task for each subject
contrasts_per_run = pd.DataFrame(tb_b_nb
                                 .groupby(['subjectkey', 'task', 'run'])
                                 .collection_id.count()
                                 .reset_index()
                                 .groupby(['task', 'run'])
                                 .collection_id.max()
                                ).rename(columns={'collection_id':'max_ncontrasts'}).reset_index()
subj_cpr = pd.DataFrame(tb_b_nb.groupby(['subjectkey', 'task', 'run'])
                        .collection_id
                        .count()
                       ).rename(columns={'collection_id':'ncontrasts'}).reset_index()
subj_cpr = subj_cpr.merge(contrasts_per_run, how='left', on=['task', 'run'])
subj_cpr = subj_cpr.loc[subj_cpr.ncontrasts == subj_cpr.max_ncontrasts].sort_values(['subjectkey', 'task', 'run'])
tb_b_nb = (subj_cpr
             .groupby(['subjectkey', 'task'])[['run']].first()
             .reset_index()
             .merge(tb_b_nb, on=['subjectkey', 'task', 'run'], how='left'))
full_data_subs = pd.unique((tb_b_nb.groupby('subjectkey')[['cr']].nunique() == tb_b_nb.groupby('subjectkey')[['cr']].nunique().max()).query('cr').index.values)
tb_b_full = tb_b_nb.loc[tb_b_nb.subjectkey.isin(full_data_subs), :]



#winsorize values
# pct_range = 0.25
# top_pct = (100-pct_range)/100
# bottom_pct = pct_range/100
# tb_b_full.loc[:, metric_cols] = tb_b_full.loc[:, metric_cols].clip(tb_b_full.quantile(bottom_pct), tb_b_full.quantile(top_pct), axis=1)

print(f'nsubs = {tb_b_full.subjectkey.nunique()}')
big_crs = []
for x,df in tb_b_full.query('run != "all"').groupby(['contrast']):
    tmp = big_sites(50, df, metric_cols).sort_values('subjectkey').reset_index()
    assert len(tmp) > 0
    big_crs.append(tmp)
big_crs[-1].groupby(['mri_info_manufacturer',
                      'mri_info_manufacturersmn','unique_scanner']).count()
assert np.array([(big_crs[0].subjectkey == bc.subjectkey).all() for bc in big_crs]).all()

In [ ]:
for crt in big_crs:
    assert (crt.index == big_crs[0].index).all()
    task = crt.task.unique()[0]
    contrast = '-'.join(crt.contrast.unique()[0].split())
    crt_file = swarm_dir/f'crt_{task}_{contrast}_beta.pkz'
    if not crt_file.exists():
        crt.to_pickle(crt_file)

# Load permutations or generate them
crt_perm_file = swarm_dir/'perms_tfmri.pkz'
if not crt_perm_file.exists():
    print("No task based permutation file found. Generating a new one.")
    # within bucket shuffle
    big_crs[0]['age_6mos'] = (big_crs[0]['interview_age'] // 6) * 6
    bin_levels = ['gender', 'ehi_ss_score', 'age_6mos']
    crt_perms = gen_binned_perms(big_crs[0], bin_levels, n_perms=2000)
    with open(crt_perm_file, 'wb') as h:
        pickle.dump(crt_perms, h)
else:
    with open(crt_perm_file.as_posix(), 'rb') as h:
        crt_perms = pickle.load(h)


# Generate bootstraps
crt_boot_file = swarm_dir/'boots_tfmri.pkz'

if not crt_boot_file.exists():
    print("No task based bootstrap file found. Generating a new one.")
    # within bucket shuffle
    bin_levels = []
    crt_boots = gen_binned_perms(big_crs[0], bin_levels, n_perms=2000, boot=True)[1:]
    with open(crt_boot_file.as_posix(), 'wb') as h:
        pickle.dump(crt_boots, h)
else:
    with open(crt_boot_file.as_posix(), 'rb') as h:
        crt_boots = pickle.load(h)
        

# If ymapper doesn't exist, create it
# We're doing this outside the swarm to ensure that every worker has the same
# encoding of the categories
crt_ymap_file = swarm_dir/'yfit_crt_beta.pkz'
if not crt_ymap_file.exists():
    crt_ymapper = DataFrameMapper([("unique_scanner",preprocessing.LabelEncoder())])
    crt_yfit = crt_ymapper.fit(big_crs[0])
    with open(crt_ymap_file, 'wb') as h:
        pickle.dump(crt_yfit, h)
else:
    with open(crt_ymap_file, 'rb') as h:
        crt_yfit = pickle.load(h)

# Write swarm files

## rs-fMRI data

In [ ]:
# TODO: Clean up paths here
from pathlib import Path
swarm_file = Path('swarm_con')
cmds = []
for pn in range(600,1000):
    out_path = '/data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/bootstrap/single_runs/bs-%04d_con_unadj.pkz'%(pn)
    cmd_base = ( 'export SINGULARITY_BINDPATH="/gs3,/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data";'
                + ' singularity exec -H ~/temp_for_singularity /data/MLDSST/singularity_images/nielsond_abcd_new-2019-04-23-76484f889f81.simg'
                + f" python /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir/run_abcd_perm_new_draws.py {pn} /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs2.0/boots_con.pkz "
                + f"/data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs2.0/yfit_con.pkz /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs2.0/big_con.pkz "
                + f"{out_path} 30 --n_draws=1 --do_ab --bootstrap; ")
    cmds.append(cmd_base)

n_draws = 25
for pn in range(600,1000):

    out_path = '/data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/bootstrap/single_runs/pn-%04d_con_unadj.pkz'%(pn)
    cmd_base = ( 'export SINGULARITY_BINDPATH="/gs3,/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data";'
                + ' singularity exec -H ~/temp_for_singularity /data/MLDSST/singularity_images//nielsond_abcd_new-2019-04-23-76484f889f81.simg'
                + f" python /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir/run_abcd_perm_new_draws.py {pn} /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs2.0/perms_con.pkz "
                + f"/data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs2.0/yfit_con.pkz /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs2.0/big_con.pkz "
                + f"{out_path} 30 --n_draws={n_draws} --do_ab; ")
    cmds.append(cmd_base)

    
    
cmds = '\n'.join(cmds)
swarm_file.write_text(cmds)

In [ ]:
cmds.split('\n')[0]

In [ ]:
# Then, outside of jupyter, run the swarm file:
!swarm -f swarm_con -g 64 -t 32 --logdir swarm_log --time 4:00:00 --partition=norm,quick --gres=lscratch:20 --module=singularity

## Task-based data

In [ ]:
crt_combinations = longtbl.query('modality == "tfmri" & metric == "beta"').groupby(['task','contrast']).size().reset_index().drop(0,axis=1).to_records(index=False)

In [ ]:
!{user_cmds['nielsond'][0]}

In [ ]:
crt_combinations

In [ ]:
users=['nielsond']
user=users[0]
user_cmds = {u:[] for u in users}
users = cycle(users)
pstep = 100
pranges = [(0,101)]
i = 0
loop_n = 4
modality='tfmri'
out_base = '/data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working'
out_subdir = 'bootstrap/single_runs'
swarm_dir = '/data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs2.0'
image = '/data/MLDSST/singularity_images/nielsond_abcd_new-2019-04-23-76484f889f81.simg'
for pr in pranges:
    cmds =[]
    for task, contrast in crt_combinations:
        cmds.extend(make_cmd_list(user, pr, modality, ['beta'], bundle=1, 
                                  task=task, contrast=contrast,
                                  out_base=out_base, out_subdir=out_subdir,
                                  swarm_dir=swarm_dir, bootstrap=True,
                                  n_draws=1, image=image, skip_completed=True))
        
#         cmds.extend(make_cmd_list(user, pr, modality, ['beta'], bundle=1, 
#                                   task=task, contrast=contrast,
#                                   out_base=out_base, out_subdir=out_subdir,
#                                   swarm_dir=swarm_dir, image=image, skip_completed=True))
        
    user = next(users)
    jobs_per_swarm = 1000
    swarm_files = []
    for ii in range(0, len(cmds), jobs_per_swarm):
        swarm_file = Path('/data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs2.0/swarm_cmds/swarm_test_%s_%d_%d_%s_%d_11'%(modality, pr[0],ii,user, loop_n))
        swarm_files.append(swarm_file.as_posix())
        swarm_file.write_text('\n'.join(cmds[ii:ii+jobs_per_swarm]))
        swarm_cmd = f'swarm -f {swarm_file} -g 20 -t auto --logdir ${{USER}}_swarm_log --time 8:00:00 --partition=norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"'
        user_cmds[user].append(swarm_cmd)
        print(len(cmds[ii:ii+jobs_per_swarm]))


In [ ]:
len(cmds)

In [ ]:
foo = make_cmd_list(user, pr, modality, ['beta'], bundle=1, 
                                  task=task, contrast=contrast,
                                  out_base=out_base, out_subdir=out_subdir,
                                  swarm_dir=swarm_dir, bootstrap=True,
                                  n_draws=1, image=image, skip_completed=True)

In [ ]:
for pr in pranges:
    cmds =[]
    for task, contrast in crt_combinations:
        cmds.extend(make_cmd_list(user, pr, modality, ['beta'], bundle=1, 
                                  task=task, contrast=contrast,
                                  out_base=out_base, out_subdir=out_subdir,
                                  swarm_dir=swarm_dir, bootstrap=True,
                                  n_draws=1, image=image, skip_completed=True))
        

In [ ]:
len(crt_combinations) * 100

In [ ]:
len([ss for ss in cmds if 'pn-' in ss ])

In [ ]:
user_cmds

In [ ]:
!swarm -f /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs2.0/swarm_cmds/swarm_test_tfmri_0_0_nielsond_4_11 -g 20 -t auto --logdir ${USER}_swarm_log --time 1:00:00 --partition=norm,quick --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"

In [ ]:
import time
for user,items in user_cmds.items():
    if user == getpass.getuser():
        for cmd in items[1:]:
            notsubmitted = True
            while notsubmitted:
                print(cmd)
                !{cmd}
                if _exit_code == 0:
                    notsubmitted=False
                else:
                    print("wait an hour and then try again")
                    time.sleep(3600)

In [ ]:
users=['nielsond']
user=users[0]
user_cmds = {u:[] for u in users}
users = cycle(users)
pstep = 100
pranges = [(0,101)]
i = 0
loop_n = 4
modality='tfmri'
out_base = '/data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working'
out_subdir = 'bootstrap/single_runs'
swarm_dir = '/data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs2.0'
image = '/data/MLDSST/singularity_images/nielsond_abcd_new-2019-04-23-76484f889f81.simg'
for pr in pranges:
    cmds =[]
    for task, contrast in crt_combinations:
#         cmds.extend(make_cmd_list(user, pr, modality, ['beta'], bundle=1, 
#                                   task=task, contrast=contrast,
#                                   out_base=out_base, out_subdir=out_subdir,
#                                   swarm_dir=swarm_dir, bootstrap=True,
#                                   n_draws=1, image=image, skip_completed=True))
        
        cmds.extend(make_cmd_list(user, pr, modality, ['beta'], bundle=1, 
                                  task=task, contrast=contrast,
                                  out_base=out_base, out_subdir=out_subdir,
                                  swarm_dir=swarm_dir, image=image, skip_completed=True))
        
    user = next(users)
    jobs_per_swarm = 1000
    swarm_files = []
    for ii in range(0, len(cmds), jobs_per_swarm):
        swarm_file = Path('/data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs/swarm_cmds/swarm_test_pns_%s_%d_%d_%s_%d_11'%(modality, pr[0],ii,user, loop_n))
        swarm_files.append(swarm_file.as_posix())
        swarm_file.write_text('\n'.join(cmds[ii:ii+jobs_per_swarm]))
        swarm_cmd = f'swarm -f {swarm_file} -g 20 -t auto --logdir ${{USER}}_swarm_log --time 8:00:00 --partition=norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"'
        user_cmds[user].append(swarm_cmd)
        print(len(cmds[ii:ii+jobs_per_swarm]))


In [ ]:
print(len(cmds))

In [ ]:
import time
for user,items in user_cmds.items():
    if user == getpass.getuser():
        for cmd in items:
            notsubmitted = True
            while notsubmitted:
                print(cmd)
                !{cmd}
                if _exit_code == 0:
                    notsubmitted=False
                else:
                    print("wait an hour and then try again")
                    time.sleep(3600)

# Code below here is not used for preregisterred analysis

# structural mri

In [ ]:
longtbl = pd.read_pickle(data_dir/'long_table.pkz')
longtbl['unique_scanner'] = longtbl.site_id_l + '__' + longtbl.mri_info_deviceserialnumber + '__' + longtbl.mri_info_softwareversion
longtbl['age_3mos'] = (longtbl['interview_age'] // 3) * 3
longtbl.drop(['subcort.aseg__wm.hypointensities.rh', 'subcort.aseg__wm.hypointensities.lh'], axis=1, inplace=True)
modality = 'smri'

In [ ]:
full_subjs = get_full_subjects(longtbl, modality)

metric_cols={}
for mm in longtbl.query('modality == @modality').metric.unique():
    metric_df = longtbl.loc[((longtbl.modality == modality) & (longtbl.metric == mm))].copy()
    unused_cols = metric_df.columns[(metric_df.notnull().sum() == 0)].values
    metric_df.drop(unused_cols, axis=1, inplace=True)

    metric_cols, meta_cols = get_cols(metric_df)
    
    metric_full = metric_df.loc[metric_df.subjectkey.isin(full_subjs), :]
    
    metric_bs = big_sites(100, metric_full, metric_cols).sort_values('subjectkey').reset_index(drop=True)
    
    metric_file = swarm_dir/f'{modality}_{mm}.pkz'
    if True:#not metric_file.exists():
            metric_bs.to_pickle(metric_file)

    # Load permutations or generate them
    modality_perm_file = swarm_dir/f'perms_{modality}.pkz'
    if not modality_perm_file.exists():
        print("No permutation file found. Generating a new one.")
        # within bucket shuffle
        bin_levels = ['gender', 'ehi_ss_score', 'age_3mos']
        modality_perms = gen_binned_perms(metric_bs, bin_levels, n_perms=2000)
        with open(modality_perm_file, 'wb') as h:
            pickle.dump(modality_perms, h)
    else:
        with open(modality_perm_file.as_posix(), 'rb') as h:
            modality_perms = pickle.load(h)


In [ ]:
#users=['nielsond']
#user_cmds = {u:[] for u in users}
#users = cycle(users)
#pstep = 100
#pranges = [(100, 200)]
i = 0
loop_n = 4
modality = 'smri'
for pr in pranges:
    user = next(users)
    cmds = make_cmd_list(user, pr, modality, longtbl.query('modality == @modality').metric.unique(), bundle=1,
                        out_base='/data/MLDSST/abcd/release11',
                        swarm_dir='/data/MLDSST/abcd/abcd_swarm_dir')
    jobs_per_swarm = 1000
    swarm_files = []
    for ii in range(0, len(cmds), jobs_per_swarm):
        swarm_file = Path('/data/MLDSST/abcd/abcd_swarm_dir/swarm_test_%s_%d_%d_%s_%d_11'%(modality, pr[0],ii,user, loop_n))
        swarm_files.append(swarm_file.as_posix())
        swarm_file.write_text('\n'.join(cmds[ii:ii+jobs_per_swarm]))
        swarm_cmd = f'swarm -f {swarm_file} -g 12 -t auto --logdir ${{USER}}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"'
        user_cmds[user].append(swarm_cmd)
        print(len(cmds[ii:ii+jobs_per_swarm]))

In [ ]:
for user,items in user_cmds.items():
    if user == getpass.getuser():
        for cmd in items:
            print(cmd)
            ! {cmd}

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

metric='t2w.white02'
roi='cort.desikan__caudalmiddlefrontal.lh'
idf = longtbl.query('metric == @metric').loc[:, ['mri_info_manufacturer', roi,'interview_age']]
g_dat=idf.query("mri_info_manufacturer == 'GE MEDICAL SYSTEMS'").loc[idf[roi].notnull(), roi]
s_dat=idf.query("mri_info_manufacturer == 'SIEMENS'").loc[idf[roi].notnull(), roi]
p_dat=idf.query("mri_info_manufacturer == 'Philips Medical Systems'").loc[idf[roi].notnull(), roi]

sns.distplot(g_dat, label='GE')
sns.distplot(s_dat, label='Philips')
sns.distplot(p_dat, label='Siemens')

plt.legend()

# dmri

In [ ]:
longtbl = pd.read_pickle(data_dir/'long_table.pkz')
longtbl['unique_scanner'] = longtbl.site_id_l + '__' + longtbl.mri_info_deviceserialnumber + '__' + longtbl.mri_info_softwareversion
longtbl['age_3mos'] = (longtbl['interview_age'] // 3) * 3
modality = 'dmri'

In [ ]:
full_subjs = get_full_subjects(longtbl, modality, metric_range={'rsi.n0.gwc':4})

metric_cols={}
for mm in longtbl.query('modality == @modality').metric.unique():
    metric_df = longtbl.loc[((longtbl.modality == modality) & (longtbl.metric == mm))].copy()
    unused_cols = metric_df.columns[(metric_df.notnull().sum() == 0)].values
    metric_df.drop(unused_cols, axis=1, inplace=True)

    metric_cols, meta_cols = get_cols(metric_df)
    
    metric_full = metric_df.loc[metric_df.subjectkey.isin(full_subjs), :]
    
    metric_bs = big_sites(100, metric_full, metric_cols).sort_values('subjectkey').reset_index(drop=True)
    
    metric_file = swarm_dir/f'{modality}_{mm}.pkz'
    if True:#not metric_file.exists():
        metric_bs.to_pickle(metric_file)

    # Load permutations or generate them
    modality_perm_file = swarm_dir/f'perms_{modality}.pkz'
    if not modality_perm_file.exists():
        print("No permutation file found. Generating a new one.")
        # within bucket shuffle
        bin_levels = ['gender', 'ehi_ss_score', 'age_3mos']
        modality_perms = gen_binned_perms(metric_bs, bin_levels, n_perms=2000)
        with open(modality_perm_file, 'wb') as h:
            pickle.dump(modality_perms, h)
    else:
        with open(modality_perm_file.as_posix(), 'rb') as h:
            modality_perms = pickle.load(h)


In [ ]:
users=['nielsond']
#user_cmds = {u:[] for u in users}
users = cycle(users)
pstep = 100
pranges = [(0,101)]
i = 0
loop_n = 4
modality = 'dmri'
for pr in pranges:
    user = next(users)
    cmds = make_cmd_list(user, pr, modality, longtbl.query('modality == @modality').metric.unique(), bundle=1)
    #cmds = make_cmd_list(user, pr, modality, ['rsi.n0.gwc'], bundle=1)
    jobs_per_swarm = 1000
    swarm_files = []
    for ii in range(0, len(cmds), jobs_per_swarm):
        swarm_file = Path('/data/nielsond/abcd/nielson_abcd_2018/swarm_dir/swarm_test_%s_%d_%d_%s_%d_11'%(modality, pr[0],ii,user, loop_n))
        swarm_files.append(swarm_file.as_posix())
        swarm_file.write_text('\n'.join(cmds[ii:ii+jobs_per_swarm]))
        swarm_cmd = f'swarm -f {swarm_file} -g 12 -t auto --logdir ${{USER}}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"'
        user_cmds[user].append(swarm_cmd)
        print(len(cmds[ii:ii+jobs_per_swarm]))

In [ ]:
export SINGULARITY_BINDPATH="/gs3,/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data"; singularity exec -H ~/temp_for_singularity /data/ML
DSST/singularity_images/abcd_tmp-2018-04-26-a7f5b7d6a3b4.img python /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs/run_abcd_perm
_new_draws.py 0 /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs/perms_tfmri.pkz /data/MBDU/nielsond/abcd_combat/nielsond_abcd_201
8_dn_working/swarm_dir_bs/yfit.pkz /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs/crt_mid_antic.large.loss.vs.neutral_run1_beta.
pkz  /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/bootstrap_test/single_runs/pn-0000_crt_mid_antic.large.loss.vs.neutral_run1.pkz 30 --n_dr
aws=25 ;export SINGULARITY_BINDPATH="/gs3,/gs4,/gs5,/gs6,/gs7,/gs8,/gs9,/gs10,/gs11,/spin1,/scratch,/fdb,/data"; singularity exec -H ~/temp_for_singularity 
/data/MLDSST/singularity_images/abcd_tmp-2018-04-26-a7f5b7d6a3b4.img python /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs/run_a
bcd_perm_new_draws.py 0 /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs/perms_tfmri.pkz /data/MBDU/nielsond/abcd_combat/nielsond_
abcd_2018_dn_working/swarm_dir_bs/yfit.pkz /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/swarm_dir_bs/crt_mid_antic.large.loss.vs.neutral_ru
n1_beta.pkz  /data/MBDU/nielsond/abcd_combat/nielsond_abcd_2018_dn_working/bootstrap_test/single_runs/pn-0000_crt_mid_antic.large.loss.vs.neutral_run1.pkz 3
0 --n_draws=25 ;

In [ ]:
for user,items in user_cmds.items():
    if user == getpass.getuser():
        for cmd in items:
            print(cmd)
            ! {cmd}

In [ ]:
len(user_cmds['nielsond'])

In [ ]:
for user,items in user_cmds.items():
    if user == getpass.getuser():
        for cmd in items[1:]:
            notsubmitted = True
            while notsubmitted:
                print(cmd)
                !{cmd}
                if _exit_code == 0:
                    notsubmitted=False
                else:
                    print("wait an hour and then try again")
                    time.sleep(3600)

# Prep for multi user run


In [ ]:
users=['nielsond', 'adamt']
user_cmds = {u:[] for u in users}
users = cycle(users)
pstep = 50
pranges = [(0, 50),
           (50,100),
           (100,150),
           (150,200)]
i = 0
loop_n = 4
swarm_files = []
jobs_per_swarm = 1000
#modality = 'smri'
modalities=['smri', 'dmri', 'tfmri']

crt_combinations = longtbl.query('modality == "tfmri" & metric == "beta" & run != "all"').groupby(['task','contrast','run']).size().reset_index().drop(0,axis=1).to_records(index=False)
for pr in pranges:
    for modality in modalities:
        user = next(users)
        if modality != 'tfmri':
            cmds = make_cmd_list(user, pr, modality, longtbl.query('modality == @modality').metric.unique(), bundle=1,
                                out_base='/data/MLDSST/abcd/',
                                swarm_dir='/data/MLDSST/abcd/abcd_swarm_dir')
        else:
            cmds=[]
            for task, contrast, run in crt_combinations:
                cmds.extend(make_cmd_list(user, pr, modality, 'beta', bundle=2, run=run, task=task, contrast=contrast, out_base='/data/MLDSST/abcd/',
                                swarm_dir='/data/MLDSST/abcd/abcd_swarm_dir'))
            
        for ii in range(0, len(cmds), jobs_per_swarm):
            swarm_file = Path('/data/MLDSST/abcd/abcd_swarm_dir/swarm_test_%s_%d_%d_%s_%d_11'%(modality, pr[0],ii,user, loop_n))
            swarm_files.append(swarm_file.as_posix())
            swarm_file.write_text('\n'.join(cmds[ii:ii+jobs_per_swarm]))
            swarm_cmd = f'swarm -f {swarm_file} -g 12 -t auto --logdir ${{USER}}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"'
            user_cmds[user].append(swarm_cmd)
            print(len(cmds[ii:ii+jobs_per_swarm]))


In [ ]:
user_cmds['adamt']

In [ ]:
!{user_cmds['nielsond'][0]}

In [ ]:
!{user_cmds['nielsond'][1]}

In [ ]:
user_cmds['nielsond']

In [ ]:
import time
for user,items in user_cmds.items():
    if user == getpass.getuser():
        for cmd in items[2:]:
            notsubmitted = True
            while notsubmitted:
                print(cmd)
                !{cmd}
                if _exit_code == 0:
                    notsubmitted=False
                else:
                    print("wait an hour and then try again")
                    time.sleep(3600)

In [ ]:
0: 15980554, 1: 15985228

In [ ]:
user_cmds = {}
user_cmds['adamt'] = ['swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_dmri_0_0_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_dmri_0_1000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_dmri_0_2000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_smri_50_0_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_tfmri_50_0_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_tfmri_50_1000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_tfmri_50_2000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_tfmri_50_3000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_tfmri_50_4000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_tfmri_50_5000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_dmri_100_0_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_dmri_100_1000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_dmri_100_2000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_smri_150_0_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_tfmri_150_0_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_tfmri_150_1000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_tfmri_150_2000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_tfmri_150_3000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_tfmri_150_4000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"',
                     'swarm -f /data/MLDSST/abcd/abcd_swarm_dir/swarm_test_tfmri_150_5000_adamt_4_11 -g 12 -t auto --logdir ${USER}_swarm_log --time 2:30:00 --partition=quick,norm --gres=lscratch:30 --module singularity --exclusive --sbatch "--constraint=core16"']

import time
import getpass
for user,items in user_cmds.items():
    if user == getpass.getuser():
        for cmd in items[1:]:
            notsubmitted = True
            while notsubmitted:
                print(cmd)
                !{cmd}
                if _exit_code == 0:
                    notsubmitted=False
                else:
                    print("wait an hour and then try again")
                    time.sleep(3600)